<a href="https://colab.research.google.com/github/SuyeonSim/Data-analystic-Project/blob/main/01_Data_Generation_and_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **github 업로드를 위한 더미 데이터 생성**

* 💡 Note: 본 프로젝트에서 사용된 원본 데이터는 기업 내부 보안 자료이므로, 분석 로직의 재현성을 위해 원본과 유사한 분포를 가진 **더미 데이터를 생성하여 분석을 진행**했습니다.

In [1]:
pip install faker pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,618 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 117540 files and direc

In [3]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('ko_KR')

def generate_time_diverse_data(row_count=30):
    data = []

    # 1. 날짜 범위 설정
    start_date = datetime(2023, 2, 6)
    end_date = datetime(2026, 2, 6)
    date_range_days = (end_date - start_date).days

    fake_catalog = {
        "리빙": ["올라운드 세라믹 냄비", "미니멀리스트 식기세트", "우드 그레인 조리도구"],
        "가전": ["스마트 에어 써큘레이터", "무선 핸디 청소기", "에코 프렌들리 가습기"],
        "잡화": ["데일리 캔버스 토트백", "오가닉 코튼 타월세트", "프리미엄 리넨 앞치마"]
    }
    categories = list(fake_catalog.keys())

    for i in range(row_count):
        # 2. 범위 내 랜덤 날짜 생성
        random_days = random.randrange(date_range_days)
        target_date = start_date + timedelta(days=random_days)
        order_date_str = target_date.strftime("%Y%m%d") # YYYYMMDD 형식

        # 주문번호 생성 (랜덤 날짜 반영)
        base_order_no = f"{order_date_str}-{str(i+1).zfill(7)}"

        # 한 주문당 품목 수
        item_count = random.randint(1, 2)

        for j in range(item_count):
            category = random.choice(categories)
            product_name = random.choice(fake_catalog[category])
            price = random.randint(15000, 200000)
            quantity = random.randint(1, 3)

            row = {
                "주문번호": base_order_no,
                "품목별 주문번호": f"{base_order_no}-0{j+1}",
                "총 주문금액": price * quantity + 3000,
                "총 결제금액": price * quantity,
                "상품번호": random.randint(1000, 9999),
                "주문상품명": product_name,
                "주문상품명(옵션포함)": f"{product_name} (색상:랜덤)",
                "수량": quantity,
                "판매가": price,
                "수령인": fake.name(),
                "수령인 휴대전화": f"010-{random.randint(1000, 9999)}-{random.randint(1000, 9999)}",
                "주문자ID": fake.user_name()[:3] + "***"
            }
            data.append(row)

    return pd.DataFrame(data)

# 30개의 랜덤 주문 생성
df_time_diverse = generate_time_diverse_data(30)

# 날짜 순서대로 보고 싶다면 정렬 (선택 사항)
df_time_diverse = df_time_diverse.sort_values(by="주문번호").reset_index(drop=True)

# 결과 확인 (주문번호 앞자리 날짜 확인)
print("--- 생성된 데이터의 날짜 분포 (상위 10개) ---")
print(df_time_diverse[["주문번호", "주문상품명"]].head(10))

# CSV 저장
df_time_diverse.to_csv("time_diverse_dummy_data.csv", index=False, encoding='utf-8-sig')
print("\n✅ 다양한 날짜가 포함된 'time_diverse_dummy_data.csv'가 생성되었습니다.")

--- 생성된 데이터의 날짜 분포 (상위 10개) ---
               주문번호         주문상품명
0  20230215-0000017  스마트 에어 써큘레이터
1  20230215-0000017   프리미엄 리넨 앞치마
2  20230511-0000018     무선 핸디 청소기
3  20230526-0000001  스마트 에어 써큘레이터
4  20230526-0000001   데일리 캔버스 토트백
5  20230528-0000008   에코 프렌들리 가습기
6  20230530-0000016  스마트 에어 써큘레이터
7  20230606-0000006   에코 프렌들리 가습기
8  20230606-0000006   프리미엄 리넨 앞치마
9  20230715-0000024   에코 프렌들리 가습기

✅ 다양한 날짜가 포함된 'time_diverse_dummy_data.csv'가 생성되었습니다.


In [4]:
import pandas as pd

# 1. 데이터 파일 경로 설정
# 보안상 faker 라이브러리로 만든 더미 데이터로 테스트합니다.
# 실제로는 브랜드의 고객 행동 데이터가 사용되었습니다.
file_path = '/content/time_diverse_dummy_data.csv'

# 2. 데이터 로드 (Raw Data)
# 인코딩은 일반적인 한글 깨짐 방지를 위해 'utf-8-sig' 또는 'cp949'를 사용합니다.
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='cp949')

# 3. 데이터 로드 결과 확인
# 데이터의 상위 5개 행과 전체적인 정보를 확인합니다.
print("--- Data Load Success ---")
print(f"Total Rows: {len(df)}")
display(df.head())
df.info()

--- Data Load Success ---
Total Rows: 47


,주문번호,품목별 주문번호,총 주문금액,총 결제금액,상품번호,주문상품명,주문상품명(옵션포함),수량,판매가,수령인,수령인 휴대전화,주문자ID
0,20230215-0000017,20230215-0000017-01,48993,45993,4165,스마트 에어 써큘레이터,스마트 에어 써큘레이터 (색상:랜덤),1,45993,김정숙,010-6277-8370,sgi***
1,20230215-0000017,20230215-0000017-02,172780,169780,9648,프리미엄 리넨 앞치마,프리미엄 리넨 앞치마 (색상:랜덤),2,84890,고영진,010-8758-4558,iji***
2,20230511-0000018,20230511-0000018-01,244908,241908,4933,무선 핸디 청소기,무선 핸디 청소기 (색상:랜덤),3,80636,강지현,010-4013-6253,ni***
3,20230526-0000001,20230526-0000001-01,311760,308760,2012,스마트 에어 써큘레이터,스마트 에어 써큘레이터 (색상:랜덤),3,102920,김지은,010-5904-3230,uco***
4,20230526-0000001,20230526-0000001-02,154940,151940,7113,데일리 캔버스 토트백,데일리 캔버스 토트백 (색상:랜덤),2,75970,김영철,010-2752-3461,bye***


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   주문번호         47 non-null     object
 1   품목별 주문번호     47 non-null     object
 2   총 주문금액       47 non-null     int64 
 3   총 결제금액       47 non-null     int64 
 4   상품번호         47 non-null     int64 
 5   주문상품명        47 non-null     object
 6   주문상품명(옵션포함)  47 non-null     object
 7   수량           47 non-null     int64 
 8   판매가          47 non-null     int64 
 9   수령인          47 non-null     object
 10  수령인 휴대전화     47 non-null     object
 11  주문자ID        47 non-null     object
dtypes: int64(5), object(7)
memory usage: 4.5+ KB


### **데이터 전처리**

* 총 결제금액, 판매가 0원인 데이터 삭제
* 주문자ID가 Null값이라면 비회원으로 분류
* 주문번호를 활용해서 주문일자 컬럼 생성
* 객단가 분석을 위해 중복 주문번호 행 삭제한 데이터프레임 생성
* 상품 분석을 위한 데이터프레임 생성

In [5]:
import pandas as pd

# 1. 유효 데이터 필터링 (결제 및 판매가가 0인 데이터 제외)
# 실질적인 매출 분석을 위해 결제 금액과 판매가가 발생한 데이터만 추출합니다.
df_clean = df[(df['총 결제금액'] > 0) & (df['판매가'] > 0)].copy()

# 2. 고객 유형 분류 (주문자ID 기준)
# 비회원(NaN) 주문을 식별하기 위해 고객 유형 컬럼을 생성합니다.
df_clean['고객유형'] = df_clean['주문자ID'].apply(lambda x: '비회원' if pd.isna(x) else '회원')

# 3. 주문일자 컬럼 생성
# 주문번호를 활용한 주문일자 컬럼 생성 (YYYYMMDD 형식 파싱)
# 주문번호의 앞 8자리를 추출하여 datetime 객체로 변환합니다.
df_clean['주문일자'] = pd.to_datetime(df['주문번호'].astype(str).str[:8], format='%Y%m%d')

# ---------------------------------------------------------
# [A] 객단가(AOV) 분석을 위한 데이터 통합 (주문번호 기준)
# ---------------------------------------------------------
# 동일한 주문번호의 중복을 제거하고 주문당 '총 결제금액'을 유지합니다.
# '총 결제금액'이 주문 전체 금액인 경우를 가정하여 첫 번째 행만 남깁니다.
df_aov = df_clean.drop_duplicates(subset=['주문번호']).copy()

# ---------------------------------------------------------
# [B] 상품 분석을 위한 데이터 정제 (품목별 주문번호 기준)
# ---------------------------------------------------------
# 각 행이 개별 품목을 나타내므로 RAW 상태를 유지하되 클리닝된 데이터를 사용합니다.
df_product = df_clean.copy()

# 3. 전처리 결과 확인
print(f"✅ 전처리 완료 내역")
summary = pd.DataFrame({
    "항목": ["전체 Raw 데이터", "유효 주문 데이터(AOV용)", "유효 품목 데이터(상품분석용)", "비회원 주문 건수"],
    "Count": [len(df), len(df_aov), len(df_product), len(df_aov[df_aov['고객유형'] == '비회원'])]
})

display(summary)

# 데이터 샘플 확인
print("\n--- 객단가 분석용 데이터 (df_aov) ---")
display(df_aov[['주문번호', '총 결제금액', '고객유형']].head())

print("\n--- 상품 분석용 데이터 (df_product) ---")
display(df_product[['주문번호', '품목별 주문번호', '주문상품명', '판매가']].head())

✅ 전처리 완료 내역


,항목,Count
0,전체 Raw 데이터,47
1,유효 주문 데이터(AOV용),30
2,유효 품목 데이터(상품분석용),47
3,비회원 주문 건수,0



--- 객단가 분석용 데이터 (df_aov) ---


,주문번호,총 결제금액,고객유형
0,20230215-0000017,45993,회원
2,20230511-0000018,241908,회원
3,20230526-0000001,308760,회원
5,20230528-0000008,212868,회원
6,20230530-0000016,51380,회원



--- 상품 분석용 데이터 (df_product) ---


,주문번호,품목별 주문번호,주문상품명,판매가
0,20230215-0000017,20230215-0000017-01,스마트 에어 써큘레이터,45993
1,20230215-0000017,20230215-0000017-02,프리미엄 리넨 앞치마,84890
2,20230511-0000018,20230511-0000018-01,무선 핸디 청소기,80636
3,20230526-0000001,20230526-0000001-01,스마트 에어 써큘레이터,102920
4,20230526-0000001,20230526-0000001-02,데일리 캔버스 토트백,75970
